In [ ]:
#Import TensorFlow
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import datasets, layers, models, utils
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import KFold
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
import sys
import foolbox, time
import math

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB *  of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

In [ ]:
#Load CIFAR100 Dataset
#Quelle
#https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar100/load_data, abgerufen 14.Juli 2021
(x_train, y_train), (x_test, y_test) = datasets.cifar100.load_data()

# Normalize pixel values to be between 0 and 1
x_train = (x_train / 255.0)- 0.5
x_test = (x_test / 255.0)- 0.5

# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

In [ ]:
#Filter Images to get CIFAR10 without CIFAR90
truth_array = (y_train==0) | (y_train==1) | (y_train==2) | (y_train==3) | (y_train==4) | (y_train==5)| (y_train==6)| (y_train==7)| (y_train==8)| (y_train==9)
truth_array_test = (y_test==0) | (y_test==1) | (y_test==2) | (y_test==3) | (y_test==4) | (y_test==5)| (y_test==6)| (y_test==7)| (y_test==8)| (y_test==9)


truth_array = truth_array.flatten()
truth_array_test = truth_array_test.flatten()
#print(truth_array.shape)

x_train_new = x_train[truth_array]
dt = np.dtype(np.uint8)
y_train_new=np.array([], dtype=dt)


for value in y_train:
    if value < 10:
        y_train_new = np.append(y_train_new, value)

print(x_train_new.shape)
print(y_train_new.shape)

x_test_new = x_test[truth_array_test]
dt = np.dtype(np.uint8)
y_test_new=np.array([], dtype=dt)

for value in y_test:
    if value < 10:
        y_test_new = np.append(y_test_new, value)

y_train_new = y_train_new.reshape(y_train_new.shape[0], 1)
y_test_new = y_test_new.reshape(y_test_new.shape[0], 1)

#inputs and targets for kFold training set
inputs = x_train_new
targets = y_train_new

In [ ]:
#Quelle
#https://matplotlib.org/stable/tutorials/introductory/pyplot.html#sphx-glr-tutorials-introductory-pyplot-py, abgerufen 14.Juli 2021

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(y_train[i][0])
plt.show()


In [ ]:
plt.imshow(x_train_new[100])
print(y_train_new[100])

In [ ]:
#Quelle
#https://github.com/hiaghosh/Defensive-Distillation/blob/master/models/cifar10/cifar10.py, abgerufen 14.Juli 2021
initial_learning_rate = 0.01

def lr_step_decay(epoch, lr):
    drop_rate = 0.5
    epochs_drop = 10.0
    return initial_learning_rate * (drop_rate ** math.floor((1+epoch)/epochs_drop))

In [ ]:
#Quelle
#https://scikit-learn.org/stable/modules/cross_validation.html#k-fold, abgerufen 14.Juli 2021
num_folds = 10

# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)


# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

    #Create and Train the Model for CIFAR10
    #Quelle
    #Architektur CNN: https://arxiv.org/pdf/1608.04644.pdf
    #https://www.tensorflow.org/guide/keras/sequential_model, abgerufen 14.Juli 2021
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),  
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(pool_size=(2,2)),   
      tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),  
      tf.keras.layers.Conv2D(128, (3, 3), activation='relu'), 
      tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
      tf.keras.layers.Flatten(),  
      tf.keras.layers.Dense(256,activation='relu'),
      tf.keras.layers.Dropout(rate=0.5),
      tf.keras.layers.Dense(256,activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'],
    )
    
    # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    #model.summary()

    #Quelle
    #https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint, abgerufen 14.Juli 2021

    checkpoint_filepath = f'tmp/checkpoint_red{fold_no}'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)

    #Quelle
    #https://www.tensorflow.org/guide/keras/train_and_evaluate, abgerufen 14.Juli 2021

    history = model.fit(
        inputs[train], targets[train],
        epochs=50,
        batch_size=128,
        validation_data=(inputs[test], targets[test]),
        callbacks=[model_checkpoint_callback, LearningRateScheduler(lr_step_decay, verbose=1)],
    )
    
    # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(model.metrics_names)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
    
    # Save the entire model
    !mkdir -p saved_models_cifar_red_kfold
    model.save(f'saved_models_cifar_red_kfold/base_model_cifar10_red_{fold_no}')
    
    # Increase fold number
    fold_no = fold_no + 1

# == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

In [ ]:
#Quelle
#https://www.tensorflow.org/tutorials/images/transfer_learning, abgerufen 14.Juli 2021

def display_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title(title)

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title(title)
    plt.xlabel('epoch')
    plt.show()

In [ ]:
# Display training curve 
display_history(history, "CNN Model CIFAR10 Reduced")

In [ ]:
#Predict random image
y=model.predict(x_test_new)
print(np.argmax(y[1500]))
plt.imshow(x_test_new[1500], cmap=plt.cm.binary)

In [ ]:
import pandas as pd
# Evaluate all 10 models
accuracy = list()
for i in range(1,11):
    model = tf.keras.models.load_model(f'saved_models_cifar_red_kfold/base_model_cifar10_red_{i}')
    model.load_weights(f'tmp/checkpoint_red{i}')

    loss, acc = model.evaluate(x_test_new, y_test_new, verbose=2)
    print('Accuracy: {:5.2f}%'.format(100 * acc))
    accuracy.append(acc)
    print(model.predict(x_test_new).shape)
    
    df = pd.DataFrame(accuracy, columns=['Scores'])
    df.to_excel('accuracy.xlsx', sheet_name='kfold', index=False)

In [ ]:
#Import Foolbox Packages
#Quelle
#https://foolbox.readthedocs.io/en/stable/, abgerufen 14.Juli 2021
from foolbox import TensorFlowModel, accuracy, samples, Model, utils, attacks, plot
from foolbox.attacks import LinfPGD, LinfDeepFoolAttack
import pandas as pd

In [ ]:
model = tf.keras.models.load_model(f'saved_models_cifar_red_kfold/base_model_cifar10_red_10')
model.load_weights(f'tmp/checkpoint_red10')

In [ ]:
#Run an attack with foolbox
#Quelle
#https://github.com/bethgelab/foolbox, abgerufen 14.Juli 2021
preprocessing = dict()
bounds = (-0.5, 0.5)
fmodel = TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)
fmodel = fmodel.transform_bounds((-0.5, 0.5))

attack_labels = tf.convert_to_tensor(y_test_new, dtype='int64')
attack_labels = tf.reshape(attack_labels, 1000)
attack_images = tf.convert_to_tensor(x_test_new, dtype='float32')

In [ ]:
#Quelle
#https://www.kaggle.com/josephvm/generating-adversarial-examples-with-foolbox, abgerufen 14.Juli 2021
predictions = model.predict(attack_images)
orig_predictions = np.argmax(predictions, axis = 1)
print(f"Accuracy:  {np.mean(orig_predictions == attack_labels) * 100:.2f} %") # Accuracy of original images
already_correct = np.sum(orig_predictions != attack_labels) # keep track of how many were already correct

In [ ]:
#Quelle
#https://foolbox.jonasrauber.de/guide/examples.html, abgerufen 14.Juli 2021
#PGD40 mit 20 Restarts
import eagerpy as ep
attack_images = ep.astensor(attack_images)
attack_labels = ep.astensor(attack_labels)
attack = attacks.LinfPGD()
epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.15,
        0.2,
        0.25,
        0.3,
        0.5,
        1.0,
    ]

success_rate = np.zeros(len(epsilons))

loop_array = np.array_split(np.arange(attack_images.shape[0]), 10)

for i , idx in enumerate(loop_array): 
    restarts = 20

    truth_array = np.zeros(shape=(len(epsilons),len(idx)), dtype=bool)
    print('Batch: ', i)
    for k in range(restarts):
        print('Restart: ', k)
        raw, clipped, is_adv = attack(fmodel, attack_images[idx], attack_labels[idx], epsilons=epsilons)
        arr = is_adv.numpy()
        truth_array = truth_array | arr
    
    success_rate += np.sum(truth_array, axis=1)
#print(attack_images.shape)
#print('Success Rate:', success_rate / attack_images.shape[0])
    
print("Attack finished!")
print('Success Rate:', success_rate / attack_images.shape[0])
df = pd.DataFrame((success_rate / attack_images.shape[0]), columns=['Success_rate'])
df.to_excel('attack_success_rate.xlsx', sheet_name='attack', index=False)

In [ ]:
print('Versionsinformationen der Module:')
print(sys.executable)
print(sys.version)
print(sys.version_info)
!conda --version
!python --version
print('Tensorflow: ' + tf.__version__)
print('Tensorflow Datasets: ' + tfds.__version__)
print('Scikit Learn: ' + sklearn.__version__)
print('Eagerpy: ' + ep.__version__)
print('Numpy: ' + np.__version__)
print('Matplotlib: ' + matplotlib.__version__)
print('Foolbox: ' + foolbox.__version__)
print('Pandas: ' + pd.__version__)